<a href="https://colab.research.google.com/github/Jcc329/Jessica_DATA606/blob/main/Notebooks/5.2.Text_Similarity_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data 606 Capstone
## Notebook 5.2
## Phase 2 Part 1: Topic Modeling
Jessica Conroy

<b> Goal: </b> 
This Notebook aims to perform topic modeling to identify key topics and differences in those topics among the pre covid and during covid time frames. 
1. LDA and NMF (non-negative matrix factorization) from game description, reviews, tags <br>
<b>2. Text similarity analysis </b>

In [1]:
!pip install gensim==3.8.0

In [2]:
!pip install googletrans==3.1.0a0

In [3]:
!pip install transformers

In [17]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 5.7 MB/s 
     |████████████████████████████████| 1.2 MB 44.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=8f1192d82b4d0551f1a50a11bdd4f51ad4c64dcc203ce097beeda93fca41dd98
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [10]:
import torch
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertConfig, BertModel, BertTokenizer
import seaborn as sns
% matplotlib inline
np.random.seed(42);

#Import libraries
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import textwrap

import gensim
print(gensim.__version__)
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LsiModel
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import preprocess_documents
from io import TextIOWrapper

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
# print(stopwords.words('english'))


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

from googletrans import Translator

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

3.8.0
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
GameData = pd.read_csv('./FinalSteamData.csv')

# Text Similarity Analysis

Based heavily on the work done in another text similarity project found here: https://github.com/Jcc329/Text_Summarization/blob/main/notebooks/Text_Similarity_Analysis_BERT_%26_Cos_Sim.ipynb

https://seaborn.pydata.org/generated/seaborn.heatmap.html


In [26]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')


In [27]:
# GameData = pd.read_csv('./FinalGameData_postProcessing.csv')

In [28]:
#To compare across pre and during covid years, I will start by creating a matrix of siimilarities and differences based on game release year
GameData['detailed_description.1'] = GameData['detailed_description.1'].astype(str)
GameData['Top Reviews by Upvotes.1'] = GameData['Top Reviews by Upvotes.1'].astype(str)
GameData['tags.1'] = GameData['tags.1'].astype(str)
Agg_on_year = GameData.groupby(['Release Year'])[['detailed_description.1', 'Top Reviews by Upvotes.1', 'tags.1']].transform(lambda x: ' '.join(x))

In [ ]:
#Descriptions
description_embeddings = sbert_model.encode(Agg_on_year['detailed_description.1'])

In [ ]:
pairwise_similarities=cosine_similarity(description_embeddings)
pairwise_differences=euclidean_distances(description_embeddings)

In [ ]:
# Visualize similarities
fig, ax = plt.subplots(figsize=(12,10))
mask = np.zeros_like(pairwise_similarities)
mask[np.triu_indices_from(mask)] = True
pairwise_similarities
ax = sns.heatmap(
    pairwise_similarities,
    cmap="spectral", mask=mask, annot=True, square=True).set(title='Similarities in Descriptions by year')
ax.set_xticklabels(Agg_on_year['Release Year'])
ax.set_yticklabels(Agg_on_year['Release Year'])
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right');
ax.title()

In [ ]:
# Visualize Differences
import seaborn as sns
fig, ax = plt.subplots(figsize=(12,10))
ax = sns.heatmap(
    pairwise_differences,
    cmap="spectral", mask=mask, annot=True, square=True).set(title='Differences in Descriptions by year')
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right');

In [ ]:
#Reviews
Review_embeddings = sbert_model.encode(Agg_on_year['Top Reviews by Upvotes.1'])
pairwise_similarities=cosine_similarity(Review_embeddings)
pairwise_differences=euclidean_distances(Review_embeddings)

In [ ]:
# Visualize similarities
fig, ax = plt.subplots(figsize=(12,10))
mask = np.zeros_like(pairwise_similarities)
mask[np.triu_indices_from(mask)] = True
pairwise_similarities
ax = sns.heatmap(
    pairwise_similarities,
    cmap="magma", mask=mask, annot=True, square=True).set(title='Similarities in Reviews by year')
ax.set_xticklabels(Agg_on_year['Release Year'])
ax.set_yticklabels(Agg_on_year['Release Year'])
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right');
ax.title()

In [ ]:
# Visualize Differences
import seaborn as sns
fig, ax = plt.subplots(figsize=(12,10))
ax = sns.heatmap(
    pairwise_differences,
    cmap="magma", mask=mask, annot=True, square=True).set(title='Differences in Reviews by year')
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right');

In [ ]:
#Tags
Tag_embeddings = sbert_model.encode(Agg_on_year['tags.1'])
pairwise_similarities=cosine_similarity(Tag_embeddings)
pairwise_differences=euclidean_distances(Tag_embeddings)

In [ ]:
# Visualize similarities
fig, ax = plt.subplots(figsize=(12,10))
mask = np.zeros_like(pairwise_similarities)
mask[np.triu_indices_from(mask)] = True
pairwise_similarities
ax = sns.heatmap(
    pairwise_similarities,
    cmap="viridis", mask=mask, annot=True, square=True).set(title='Similarities in Tags by year')
ax.set_xticklabels(Agg_on_year['Release Year'])
ax.set_yticklabels(Agg_on_year['Release Year'])
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right');
ax.title()

In [ ]:
# Visualize Differences
import seaborn as sns
fig, ax = plt.subplots(figsize=(12,10))
ax = sns.heatmap(
    pairwise_differences,
    cmap="viridis", mask=mask, annot=True, square=True).set(title='Differences in Tags by year')
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right');

# Compare pre-covid and during covid directly

In [ ]:
preCOVID = GameData[GameData['Release Year'].isin([2018,2019])]
COVID = GameData[GameData['Release Year'].isin([2021,2022])]

In [ ]:
preCOVID_descriptions = preCOVID['detailed_description.1'].apply(lambda x: ' '.join(x))
COVID_descriptions = COVID['detailed_description.1'].apply(lambda x: ' '.join(x))

corpus = [preCOVID_descriptions, COVID_descriptions]
description_embeddings = sbert_model.encode(corpus)

In [ ]:
pairwise_similarities=cosine_similarity(description_embeddings)
pairwise_differences=euclidean_distances(description_embeddings)

print('Cosine Similarity: ' + str(pairwise_similarities[0][1]))
print('Euclidean Distance: ' + str(pairwise_differences[0][1]))